<a href="https://colab.research.google.com/github/JungCesar/bscaithesis/blob/master/final_bsc_ai_thesis_mfcc_1dconv_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dysarthria Classification Algorithm
## Features: MFCCs
## Classifier: 1D CONV

In this notebook, I will create a dyasarthria classification algorithm for my bachelor's thesis Artificial Intelligence at the University of Amsterdam. Dysarthria occurs when the muscles you use for speech are weak or you have difficulty controlling them. The title of my thesis is "Investigating Pre-Trained Self-Supervised Deep Learning Models for Disease Recognition". The idea is to apply the algorithm created here to other datasets as well, for example audiofiles from the DementiaBank databases.

The [Torgo](http://www.cs.toronto.edu/~complingweb/data/TORGO/torgo.html) dataste perfectly fits this purpose. It consists of dyasrthric male and female speakers, either caused by cerebral palsy (CP) or amyotrophic lateral sclerosis (ALS) and their non-dysarthric counterparts, also known as the healthy control group.

I will exrtact 30 MFCCs using Python's Librosa library. Then I will train a 1d convolutional network on the binary classification between 'dysarthria' and 'non-dysarthria'.

I will take a bottom-up approach, try to be as operating system-independent as possible, and explain the steps in detail below. The structure will be as follows:

1.   Installing and Exlpaining Necessary Packages
2.   Downloading, Storing, and Accessing (Loading) the Dataset
3.   Exploratory Data Analysis (EDA)
4.   Dataset Preprocessing
5.   Dataset Splitting (into a train- and test set)
6.   MFCC Feature Extraction
7.   Training 1d Convolutional Classifier
8.   Model Evaluation

Notes:

*   The above link to TORGO redirects to the Computational Linguistics website of the university of Toronto. An in-depth explanation can be found there, but the actual dataset that will be used in this notebook, comes from Kaggle. [Kaggle's version of Torgo](https://www.kaggle.com/datasets/iamhungundji/dysarthria-detection) is already some sort of preprocessed form of the original one.


Firstly, I install the necessary libraries and use `%%capture` to hide all the output.

In [1]:
%%capture
!pip3 install librosa numpy scikit-learn

Then, I mount, or say connect to, my Google Drive, where my data is being stored. This is the easiest way to access your data in Google Colab without the need of uploading everything from tuntime to runtime, but when working in a local Jupyter Notebook and having your data stored locally on your machine, one could replace these lines with simply accessing your local files.

In [2]:
# Get access to personal Google Drive account
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


As the CNN requires the input to be of a constant shape, I will a second from the exact middle of all audio files that have a duration of longer than 1 second. All files with a duration of less than one second will be discarded.

In [33]:
# str(path) returns something like: /content/drive/MyDrive/bsc-ai-thesis/torgo_data/dysarthria_female/F01_Session1_0002.wav
# tqdm is used to create a smart progress bar for the loops, for example it shows loading time
import librosa
import torchaudio
from pathlib import Path
from tqdm import tqdm

data = []

# Loop through all audio files within the TORGO database
for path in tqdm(Path("/content/drive/MyDrive/bsc-ai-thesis/torgo_data").glob("**/*.wav")):

    name = str(path).split('/')[-1].split('.')[0]
    label = str(path).split('/')[-2]

    try:
        # Set standard offset for files with duration of 1s
        offset = 0

        # Option to resample the audio  to 16kHz, same as wav2vec expects
        # Load the entire audio file
        y_temp, sr_temp = librosa.load(path=path, sr=16000)

        # Option to resample the audio  to 16kHz, same as wav2vec expects

        # Get the total duration
        total_duration = librosa.get_duration(y=y_temp, sr=sr_temp)

        if total_duration < 1:
            continue

        if total_duration > 1:
            # Calculate the offset to start from the middle of the file
            offset = (total_duration - 1) / 2

        # Load 1 second from the middle of the audio file
        y, sr = librosa.load(path=path, sr=16000, offset=offset, duration=1)

        # Try loading all files here, when broken this will jump to exception
        s = torchaudio.load(path)
        data.append({
            "filename": name,
            "path": str(path),
            "y": y,
            "sr": sr,
            "dysarthria_class": label
        })

    # Excpetions will be elaborated here
    except Exception as e:
        print("\n", e, str(path))
        pass

79it [00:00, 129.00it/s]<ipython-input-33-406dd4128d0f>:22: UserWarning: PySoundFile failed. Trying audioread instead.
  y_temp, sr_temp = librosa.load(path=path, sr=16000)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
114it [00:01, 150.30it/s]


  /content/drive/MyDrive/bsc-ai-thesis/torgo_data/dysarthria_female/F01_Session1_0068.wav


2000it [00:14, 139.66it/s]


Now, I convert the data to a pandas dataframe and show the first five instances to visually see the data that I am dealing with.

In [34]:
# Show how the Pandas dataframe looks like currently
import pandas as pd
df = pd.DataFrame(data)
df.head()

,filename,path,y,sr,dysarthria_class
0,F01_Session1_0006,/content/drive/MyDrive/bsc-ai-thesis/torgo_dat...,"[0.0011291504, 0.00036621094, 0.0019836426, 0....",16000,dysarthria_female
1,F01_Session1_0038,/content/drive/MyDrive/bsc-ai-thesis/torgo_dat...,"[0.13388062, 0.09274292, 0.10745239, 0.1221313...",16000,dysarthria_female
2,F01_Session1_0015,/content/drive/MyDrive/bsc-ai-thesis/torgo_dat...,"[-0.0020446777, -0.0021362305, -0.0021972656, ...",16000,dysarthria_female
3,F01_Session1_0024,/content/drive/MyDrive/bsc-ai-thesis/torgo_dat...,"[-0.0012512207, -0.0011291504, -0.0011901855, ...",16000,dysarthria_female
4,F01_Session1_0053,/content/drive/MyDrive/bsc-ai-thesis/torgo_dat...,"[-0.00064086914, -0.00048828125, -0.0004577636...",16000,dysarthria_female


Next, my goal is to train a model to recognize the presence or absence of dysarthria in speech, it would be appropriate to combine the four data folders into two categories: patients *with* the disease and patients *without* the disease. This will simplify the training process and ensure that the model is focused on recognizing the presence of dysarthria, instead of recognizing a gender. Let's see how many audio files each of the now two categories contain.

It is noticeable that there was one instance of audio filtered out previously, specifically an instance of 'dysarthria'. The 1000 samples for each class, were reduced to 999 for 'dysarthria'.

In [35]:
# Eliminate difference between male and female and print distribbution
df = df.replace({'dysarthria_class' : {'dysarthria_female': 'dysarthria', 'dysarthria_male': 'dysarthria', 'non_dysarthria_female': 'non_dysarthria', 'non_dysarthria_male': 'non_dysarthria'}})
print("Labels: ", df["dysarthria_class"].unique())
print()
df.groupby('dysarthria_class').count()

Labels:  ['dysarthria' 'non_dysarthria']



,filename,path,y,sr
dysarthria_class,,,,
dysarthria,970,970,970,970
non_dysarthria,997,997,997,997


Once again, I want to see how the data in the 'path' column looks like.

In [36]:
print(df['path'][0])

/content/drive/MyDrive/bsc-ai-thesis/torgo_data/dysarthria_female/F01_Session1_0006.wav


For my classification problem, padding, i.e. adding a specific number or set of numbers to the front or back of your audio sample to reach a specific length, is considered unnatural and therefore not desired. That is why I will remove all audio files that durate less than a second (1s or 1000ms).

In [37]:
import librosa

# Count how many samples there are of less than 1 second long
# In the next section these will be removed
lstsmall = []
lstlarge = []

for i in range(len(df['path'])):
    if librosa.get_duration(path=df['path'][i]) < 1:
        lstsmall.append(i)
    else:
        lstlarge.append(i)

print(len(lstsmall), lstsmall)
print(len(lstlarge), lstlarge)

# Removing all audio files with a  duration of less than a second (1s or 1000ms)
print(f"Step 0: {len(df)}")

# Audio files with length of less than 1 second cause error for MFCC extraction
# Therefore delete these files
df["status"] = df["path"].apply(lambda x: True if (librosa.get_duration(path=x) > 1) else None)
df = df.dropna(subset=["status"])
df = df.drop(columns="status")

print(f"Step 1: {len(df)}")

0 []
1967 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219,

In [38]:
print(str(df["y"][0].shape))

(16000,)


In [50]:
print(len(X))
print(X[0].shape)

1967
(30, 32)


In [41]:
for f in df["y"]:
    if str(f.shape) != "(16000,)":
        print("err")

In [42]:
import numpy as np
# from sklearn.model_selection import train_test_split

# Function to extract the MFCCs of a specific files, specified by its path
def extract_mfcc(filename):
    y, sr  = librosa.load(filename)

    # mfcc = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=30).T, axis=0)
    # return mfcc

    # What is the difference of taking the mean of the transpose here instead of just keeping MFCC?????
    # Each coefficient will be averaged over time.
    # Make sure successive windows overlap slightly
    # mfcc = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20, n_fft=512, center=True).T, axis=0)

    # mfcc = np.mean(librosa.feature.mfcc(y=y, sr=sr).T, axis=0)
    # return mfcc

    return librosa.feature.mfcc(y=y, sr=sr, n_mfcc=30, n_fft=512)
    # return mfcc

In [43]:
test_size=0.2

X, y = [], []
# df = df.reset_index()  # make sure indexes pair with number of rows
for _, row in df.iterrows():
    # Get the MFCC for all audio files
    features = librosa.feature.mfcc(y=row['y'], sr=row['sr'], n_mfcc=30, n_fft=512)

    # Get the corresponding label: 'dysarthria' or 'non_dysarthria'
    label = row['dysarthria_class']

    # add to data
    X.append(features)
    y.append(label)

In [47]:
# print(X)
# for x in X:
#     print(x.shape)

print(type(X))

# print(X[0])
# print(type(X[0]))
# print(len(X[0]))
# print(X[0][0])
# print(type(X[0][0]))
# print(len(X[0][0]))

print(X[0].shape)

# for x in X:
#     if str(X.shape) != "(30, 44)":
#         print("err")

<class 'list'>
(30, 32)


In [48]:
print(len(y))
print(type(y))

1967
<class 'list'>


In [54]:
from sklearn.preprocessing import LabelEncoder

# Transform labels to integers
le = LabelEncoder()
y = le.fit_transform(y)  # this will transform 'non_dysarthria' to 0 and 'dysarthria' to 1

In [60]:
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

# Assuming you have your features in X and labels in y
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert the splits to PyTorch tensors
train_dataset = TensorDataset(torch.tensor(X_train), torch.tensor(y_train))
test_dataset = TensorDataset(torch.tensor(X_test), torch.tensor(y_test))

batch_size = 32
# Create data loaders
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
testloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [89]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.nn import functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        # Convolutional layer with 30 input channels, 32 output channels, and a kernel size of 3
        self.conv1 = nn.Conv1d(in_channels=30, out_channels=32, kernel_size=3)

        # Maxpool layer with size 2
        self.pool = nn.MaxPool1d(2)

        # Convolutional layer with 32 input channels, 64 output channels, and a kernel size of 3
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3)

        # Fully connected layer with input size that will be determined later, and output size of 2 (since we have two classes to predict)
        self.fc1 = nn.Linear(in_features=384, out_features=2)  # in_features needs to be updated depending on your data and layers configuration.

        # Dropout layer with p=0.5 (probability of an element to be zeroed)
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))

        # Flatten the output for each sample
        x = x.view(x.size(0), -1)

        # Apply dropout
        x = self.dropout(x)

        # print("Shape before FC layer: ", x.shape)  # add this line to check the shape

        # Pass the output through the fully connected layer
        x = self.fc1(x)

        return x

# Step 3: Define a loss function and optimizer
model = Net()
criterion = nn.CrossEntropyLoss()
# lr=0.001, lr=3e-05
# optimizer = optim.SGD(model.parameters(), lr=3e-05, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08)

# Step 4: Train the network
num_epochs = 30
for epoch in range(num_epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 20 == 19:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

Finished Training


In [98]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def evaluate_model(model, data_loader):
    model.eval()  # Set the model to evaluation mode
    y_true = []
    y_pred = []

    # .to(deivce)
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs = inputs
            labels = labels

            outputs = model(inputs)  # Forward pass
            _, predicted = torch.max(outputs.data, 1)  # Get predictions

            y_true.extend(labels.tolist())
            y_pred.extend(predicted.tolist())

    # Calculate metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='binary')
    recall = recall_score(y_true, y_pred, average='binary')
    f1 = f1_score(y_true, y_pred, average='binary')
    auc = roc_auc_score(y_true, y_pred)

    print(f'Accuracy: {100*accuracy:.2f}\nPrecision: {100*precision:.2f}\nRecall: {100*recall:.2f}\nF1-score: {100*f1:.2f}\nAUC-ROC: {100*auc:.2f}')

# Evaluate the model on the test set
evaluate_model(model, testloader)

Accuracy: 90.10
Precision: 85.09
Recall: 97.49
F1-score: 90.87
AUC-ROC: 90.03


In [90]:
# Test the model
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in testloader:
        output = model(inputs)
        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Test Accuracy: {100 * correct / total}')

Test Accuracy: 89.08629441624366


In [ ]:
# from sklearn.metrics import accuracy_score # to measure how good we are

# # harmonic mean of the precision and recall, where an F1 score reaches its best value at 1 and worst score at 0
# from sklearn.metrics import f1_score # F1 = 2 * (precision * recall) / (precision + recall)
# from sklearn.metrics import roc_auc_score

# # predict 25% of data to measure how good we are
# y_pred = model.predict(X_test)

# # calculate the accuracy
# accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)
# print("Accuracy: {:.2f}%".format(accuracy*100))

# f1 = f1_score(y_true=y_test, y_pred=y_pred, labels=['dysarthria', 'non_dysarthria'], average='binary', pos_label='dysarthria')
# print(f"f1: {f1}")

# y_proba = model.predict_proba(X_test)

# roc_auc = roc_auc_score(y_true=y_test,  y_score=y_proba[:, 1])
# print(f"ROC AUC: {roc_auc}")

In [ ]:
# def load_data(test_size=0.2):
#     X, y = [], []
#     df = data.reset_index()  # make sure indexes pair with number of rows
#     for _, row in df.iterrows():
#         # print(row['path'], row['dysarthria_class'])
#         # get the base name of the audio file
#         # basename = os.path.basename(file)
#         # extract speech features
#         features = extract_mfcc(row['path'])
#         # get the emotion label
#         label = row['dysarthria_class']
#         # add to data
#         X.append(features)
#         y.append(label)
#     # split the data to training and testing and return it
#     return train_test_split(np.array(X), y, test_size=test_size)